In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates. All rights reserved.

# Render a textured mesh

This tutorial shows how to:
- load a mesh and textures from an `.obj` file.
- set up a renderer
- render the mesh
- vary the rendering settings such as lighting and camera position
- use the batching features of the pytorch3d API to render the mesh from different viewpoints

## 0. Install and Import modules

Ensure `torch` and `torchvision` are installed. If `pytorch3d` is not installed, install it using the following cell:

In [17]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith(("1.13.", "2.0.")) and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

In [24]:
import os
import torch
import matplotlib.pyplot as plt

# Util function for loading meshes
from pytorch3d.io import load_objs_as_meshes, load_obj

# Data structures and functions for rendering
from pytorch3d.structures import Meshes,Pointclouds
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
from pytorch3d.vis.texture_vis import texturesuv_image_matplotlib
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras,
    PointLights,
    DirectionalLights,
    Materials,
    RasterizationSettings,
    MeshRenderer,
    MeshRasterizer,
    SoftPhongShader,
    TexturesUV,
    TexturesVertex
)

# add path for demo utils functions
import sys
import os
sys.path.append(os.path.abspath(''))

If using **Google Colab**, fetch the utils file for plotting image grids:

In [25]:
!wget https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/plot_image_grid.py
from plot_image_grid import image_grid

--2023-08-10 04:33:49--  https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/plot_image_grid.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘plot_image_grid.py’

plot_image_grid.py  100%[===================>]   1.57K  --.-KB/s    in 0.001s  

2023-08-10 04:33:49 (1.98 MB/s) - ‘plot_image_grid.py’ saved [1608/1608]



OR if running **locally** uncomment and run the following cell:

In [ ]:
# from utils import image_grid

In [4]:
!ls

plot_image_grid.py  __pycache__  sample_data


In [5]:
from google.colab import drive

drive.mount('/content/drive')

import os

os.chdir("/content/drive/MyDrive/Colab Notebooks")


Mounted at /content/drive


In [19]:
!ls

1_b.gif			      q_5-2.gif
2.1.gif			      q_5-3.gif
3_b.gif			      rendering_generic_3d_representations.ipynb
data			      rendering_parametric_and_implicit.ipynb
deform_form.ipynb	      rendering_point_clouds_from_rgbd.ipynb
plotly_rendered_meshes.ipynb  rendering_texture_obj.ipynb
q_5-1_pc1.gif		      render_tetrahedron_and_retexturing.ipynb
q_5-1_pc2.gif		      render_texture_mesh_and_dolly_zoom.ipynb
q_5-1_pc_union.gif	      sample_pointcloud_from_mesh.ipynb


In [7]:
if torch.cuda.is_available():
  device=torch.device("cuda:0")
else:
  device=torch.device("cpu")
  print("gpu is not available")


In [8]:
!ls

1_b.gif			      q_5-2.gif
2.1.gif			      q_5-3.gif
3_b.gif			      rendering_generic_3d_representations.ipynb
data			      rendering_parametric_and_implicit.ipynb
deform_form.ipynb	      rendering_point_clouds_from_rgbd.ipynb
plotly_rendered_meshes.ipynb  rendering_texture_obj.ipynb
q_5-1_pc1.gif		      render_tetrahedron_and_retexturing.ipynb
q_5-1_pc2.gif		      render_texture_mesh_and_dolly_zoom.ipynb
q_5-1_pc_union.gif	      sample_pointcloud_from_mesh.ipynb


In [26]:
def get_points_renderer(
    image_size=512, device=None, radius=0.01, background_color=(1, 1, 1)
):
    """
    Returns a Pytorch3D renderer for point clouds.

    Args:
        image_size (int): The rendered image size.
        device (torch.device): The torch device to use (CPU or GPU). If not specified,
            will automatically use GPU if available, otherwise CPU.
        radius (float): The radius of the rendered point in NDC.
        background_color (tuple): The background color of the rendered image.

    Returns:
        PointsRenderer.
    """
    if device is None:
        if torch.cuda.is_available():
            device = torch.device("cuda:0")
        else:
            device = torch.device("cpu")
    raster_settings = PointsRasterizationSettings(image_size=image_size, radius=radius,)
    renderer = PointsRenderer(
        rasterizer=PointsRasterizer(raster_settings=raster_settings),
        compositor=AlphaCompositor(background_color=background_color),
    )
    return renderer

In [30]:
import random
import math
from tqdm.notebook import tqdm
from PIL import Image, ImageDraw
import imageio
import numpy as np


def PCD_from_mesh(
    image_size=512,
    num_frames=100,
    duration=3,
    device=None,
    output_file="output/",
):
    if device is None:
        device = device

    verts, faces, aux = load_obj("data/cow_mesh/cow.obj")
    faces = faces.verts_idx

    # Sample a face with probability proportional to the area of the face
    num_triangle = faces.shape[0]
    areas = torch.zeros((num_triangle))
    for i in range(num_triangle):
        v1 = verts[faces[i][0]][:]
        v2 = verts[faces[i][1]][:]
        v3 = verts[faces[i][2]][:]
        areas[i] = abs(0.5 * torch.inner(v1-v2, v1-v3))

    weight = areas / sum(areas)

    num_samples = 1000 # number of point cloud
    sampled_faceidx = []
    for i in range(num_samples):
      rnd = random.uniform(0, 1)
      for j, w in enumerate(weight):
          if w<0:
              raise ValueError("Negative weight encountered.")
          rnd -= w
          if rnd < 0:
              sampled_faceidx.append(j)
              break

    print("number of samples generated is:", num_samples)
    if num_samples != len(sampled_faceidx): raise ValueError("WTF?")

    points = torch.zeros((num_samples,3))
    for i in range(num_samples):
      idx = sampled_faceidx[i]
      p1, p2, p3 = verts[faces[idx][0]][:], verts[faces[idx][1]][:], verts[faces[idx][2]][:]
      alpha = random.uniform(0, 1)
      alpha2 = random.uniform(0, 1)
      alpha1 = 1- math.sqrt(alpha)
      v = alpha1 * p1 + (1-alpha1)*alpha2*p2 + (1-alpha1)*(1-alpha2)*p3
      points[i] = v

    color = (points - points.min()) / (points.max() - points.min())


    cow_point_cloud = Pointclouds(
        points=[points], features = [color]
    )

    print(points.shape)
    print(color.shape)
    print(cow_point_cloud)

    renders = []
    angles = np.linspace(0,360,num_frames)
    for i, angle in enumerate(tqdm(angles)):
        R, T = pytorch3d.renderer.look_at_view_transform(dist=5.0, elev=2, azim=angle)
        cameras = pytorch3d.renderer.FoVPerspectiveCameras(R=R, T=T, device=device)
        renderer = get_points_renderer(image_size=image_size, device=device)
        rend = renderer(cow_point_cloud, cameras=cameras)
        rend = rend.cpu().numpy()[0, ..., :3]  # (B, H, W, 4) -> (H, W, 3)
        renders.append(rend)

    images = []
    for i, r in enumerate(renders):
        image = Image.fromarray((r * 255).astype(np.uint8))
        draw = ImageDraw.Draw(image)
        draw.text((20, 20), f"angle: {angles[i]:.2f}", fill=(0, 0, 255))
        images.append(np.array(image))
    imageio.mimsave(output_file, images, fps=(num_frames / duration))




PCD_from_mesh()


number of samples generated is: 1000
torch.Size([1000, 3])
torch.Size([1000, 3])


  0%|          | 0/100 [00:00<?, ?it/s]

AttributeError: ignored